In [34]:
import numpy as np
from scipy.optimize import fsolve  
from scipy.integrate import quad 

# Task 2: Computing Peng-Robinson EOS

Give our input conditions and the gas-specific constants that we are going to use.

In [35]:
#Input conditions
## intial pressure and volume
P1 = 100 # in bar
T1 = 26.85 # in Celsius

## final pressure and volume
P2 = 40 # in bar
T2 = -28.15 # in Celsius

Since we would be looking at different species besides water, we elected to include a prototype that will have the critical points and important properties for water and N$_2$. Use critical constants values from Appendix A of Koretsky.

In [36]:
species = 'N2' # enter the name of the species here
def properties(species):
    """Data for the critical temperature and pressure of a species"""
    if species in np.array(['H2O','water']):
        T_c = 647.3 # critical temperature of Water (in Kelvin)
        P_c = 220.48e5 # critical pressure of Water (in Pascals)
        MW = 18e-3 #Molecular weight of Water kg/gmol 
        w = 0.344 #Pitzer acentric factor of Water (important for Peng-Robinson EOS)
    elif species in np.array(['N2']):
        T_c = 126.2 #K
        P_c = 33.84e5 #Pa
        MW = 28.0134e-3 #Molecular weight of N2 kg/mol 
        w = 0.039 #Pitzer acentric factor
    return (T_c, P_c, w, MW)

# Peng-Robinson Equation of State:
$$P = \frac{RT}{v-b} -\frac{a \alpha}{v^2 + 2bv - b^2}$$
where 
$$ a = 0.45724 \frac{\left(R T_c \right)^2}{P_c}, $$

$$ b = 0.07780 \frac{R T_c}{P_c}, $$

$$ \alpha = \left[ 1 + \kappa \left(1 - \sqrt{\frac{T}{T_c}} \right)\right]^2, $$

$$ \kappa = 0.37464 + 1.54226 \omega - 0.26992 \omega^2 $$

In [37]:
def P_PR(T, v, species):
    """Compute the pressure from Peng-Robinson EOS"""
    T_c, P_c, w, MW  = properties(species)
    T_K = T + 273.15 # temperature converted from Celcius to Kelvin
    R_gas = 8.314 # gas constant in J K^-1 mol^-1
    a = 0.45724 * (R_gas * T_c)**2 / P_c
    b = 0.07780 * R_gas * T_c / P_c
    T_r = T_K / T_c # reduced temperature
    kappa = 0.37464 + 1.54226 * w - 0.26992 * w**2
    alpha = (1 + kappa * (1 - np.sqrt(T_r)))**2
    P = (R_gas * T_K)/(v - b) - a * alpha / (v**2 + 2 * b * v - b**2) # in Pa
    return P

# Ideal Gas Law

$$ v = \frac{R T}{P} $$

We will need this later to determine a good guess for $v$ using the given conditions.

In [38]:
def v_ideal(P,T):
    """Molar volume from the ideal gas law"""
    R_gas = 8.314 # gas constant in J K^-1 mol^-1
    P_Pa = P * 1e5 # convert bar to Pa
    T_K = 273.15 + T
    v = R_gas * T_K / P_Pa # in m^3/mol
    return v 

# Solving for the molar volume

Now we will solve for volume using fsolve. Let us define a function that solves for Peng-Robinson EOS given the value of the pressure, molar volume, and temperature 

In [39]:
def func(v, P, T, species = species): # While writing equation of state functions, follow the sequenece: v,P,T
    RHS = P_PR(T, v, species) # pressure from Peng-Robinson EOS, given the T and v
    LHS = P*1e5 # given pressure in Pa
    return LHS-RHS

Now we will solve for $v$ using Peng-Robinson EOS. We will use the $v$ from the ideal gas law as a guess

In [40]:
args1 = (P1,T1) # additional arguments for the function 
                # (NOTE: The sequence of writing the additional arguments 
                # should be same as you have written in function definition)
        
v01 = v_ideal(*args1) # initial guess for volume 
                      # (NOTE: It is important to use correct initial guess 
                      # to get the volume. For starters, use Ideal gas volume 
                      # as an initial guess)

# volume for state 1
v1= fsolve(func,v01,args1)

# volume for state 2
args2 = (P2,T2)
v02 = v_ideal(*args2)
v2 = fsolve(func,v02,args2)

# Print Results

We will need to print $v$. Here is a code that will do it (no need to change it):

In [41]:
def print_specific_volume(v, T, p, species):
    """Print the specific volume, temperature, and pressure of the species"""
    T_c, P_c, w, MW  = properties(species)
    print(f'Results for {species}:')
    num_lin = 74 * '-'
    print(num_lin)
    print('| State | Temperature (C) | Pressure (bar) | Specific Volume (m^3 kg^-1) |')
    print(num_lin)
    for l in range(T.shape[0]):
        print(f'|{l + 1:^7.0f}|{T[l]:^17.2f}|{p[l]:^16.2f}|{v[l]/MW:^29.6f}|')
    print(num_lin)
    return None
v_array = np.array([v1[0], v2[0]])
T_array = np.array([T1, T2])
p_array = np.array([P1, P2])
print_specific_volume(v_array, T_array, p_array, species)

Results for N2:
--------------------------------------------------------------------------
| State | Temperature (C) | Pressure (bar) | Specific Volume (m^3 kg^-1) |
--------------------------------------------------------------------------
|   1   |      26.85      |     100.00     |          0.008807           |
|   2   |     -28.15      |     40.00      |          0.017418           |
--------------------------------------------------------------------------


Check your results with the values in Steam tables. They should be very similar or match. 

# Departure functions for the Peng-Robinson EOS
Departure functions:

$$\displaystyle \Delta h' (T,P) = -\int_v^\infty \left[ T \left(\frac{\partial P}{\partial T}\right)_v +v \left(\frac{\partial P}{\partial v}\right)_T \right] dv$$

$$\displaystyle \Delta s' (T,P) =  -\int_v^\infty \left[ 
 \left( \frac{\partial P}{\partial T} \right)_v - \left(\frac{R}{v}\right) \right] dv + R \ln \left( \frac{Pv}{RT}\right)$$

Using the Peng-Robinson EOS, we can derive $\left( \frac{\partial P}{\partial T} \right)_v$ and $\left(\frac{\partial P}{\partial v}\right)_T$. They are:
$$ \left( \frac{\partial P}{\partial T} \right)_v  = \frac{R}{v - b} - \left( \frac{a}{v^2 + 2vb - b^2} \right) \frac{\partial \alpha}{\partial T}$$
$$ \frac{\partial \alpha}{\partial T} = 2 \left(1 + \kappa \left[ 1 - \sqrt{\frac{T}{T_c}} \right] \right) \left( -\frac{\kappa}{2 T_c} \sqrt{\frac{T_c}{T}} \right) = -\frac{\kappa}{T_c} \sqrt{\alpha \frac{T_c}{T}} = - \kappa \sqrt{\frac{\alpha}{T T_c}}$$
$$ \left(\frac{\partial P}{\partial v}\right)_T = -\left[ \frac{RT}{(v - b)^2} - \frac{2a\alpha(v + b)}{(v^2 + 2bv - b^2)^2} \right]$$


The $\Delta h' (T,P)$ and $\Delta s' (T,P)$ will be solved via numerical integration. 

Two seperate functions will be created for each integrand, and one function will be created to compute $\Delta h' (T,P)$ and $\Delta s' (T,P)$ per the user's request. Code will vary for each student, but final answer should be the same.

In [42]:
def dhprim_integrand(v, T, species):
    """Intgrand for the Departure Function for enthalpy"""
    T_c, P_c, w, MW  = properties(species)
    R_gas = 8.314 
    a = 0.45724 * (R_gas * T_c)**2 / P_c
    b = 0.07780 * R_gas * T_c / P_c
    T_r = T / T_c # reduced temperature
    kappa = 0.37464 + 1.54226 * w - 0.26992 * w**2
    alpha = (1 + kappa * (1 - np.sqrt(T_r)))**2
    
    # derivatives 
    dalpha_dT = -kappa * np.sqrt(alpha / (T_r)) / T_c
    dP_dT = R_gas / (v - b) - dalpha_dT * (a / (v**2 + 2 * v * b - b**2))
    dP_dv = - (R_gas * T / (v - b)**2 - (2 * a * alpha * (v + b)) / (v**2 + 2 * v * b - b**2)**2)
    
    # integrand 
    integrand = dP_dT * T + dP_dv * v # in J/mol
    return integrand

def dsprim_integrand(v, T, species):
    """Integrand for the Departure Function for entropy"""
    T_c, P_c, w, MW  = properties(species)
    R_gas = 8.314 
    a = 0.45724 * (R_gas * T_c)**2 / P_c
    b = 0.07780 * R_gas * T_c / P_c
    T_r = T / T_c # reduced temperature
    kappa = 0.37464 + 1.54226 * w - 0.26992 * w**2
    alpha = (1 + kappa * (1 - np.sqrt(T_r)))**2
    
    # derivatives 
    dalpha_dT = -kappa * np.sqrt(alpha / T_r) / T_c
    dP_dT = R_gas / (v - b) - dalpha_dT * (a / (v**2 + 2 * v * b - b**2))
    
    # integrand 
    integrand = dP_dT - R_gas / v # in J/mol
    return integrand

def dh_ds_prim(v, T, P, species):
    """Departure function values for dh' and ds'"""
    T_K = T + 273.15
    P_Pa = P * 1e5 # in Pa
    arguments = (T_K, species)
    R_gas = 8.314
    
    # Dh departure function
    dh_prime = -quad(dhprim_integrand, v, np.inf, args=arguments)[0]
    
    # Ds departure function
    ds_integral = quad(dsprim_integrand, v, np.inf, args=arguments)[0]
    ds_prime = -ds_integral + R_gas * np.log(P_Pa * v / (R_gas * T_K))
    return (dh_prime, ds_prime)

# Print the results

In [43]:
def partion_func_results(v, T, p, species):
    """Print results for the entropy and enthalpy departure functions"""
    h_array = []
    s_array = []
    print(f'Results for {species}:')
    num_lin = 48 * '-'
    print(num_lin)
    print("| State | Dh' (J mol^-1) | Ds' (J K^-1 mol^-1) |")
    print(num_lin)
    for l in range(T.shape[0]):
        h, s = dh_ds_prim(v[l], T[l], p[l], species)
        h_array += h,
        s_array += s,
        print(f'|{l + 1:^7.0f}|{h:^16.3f}|{s:^21.3f}|')
    print(num_lin)
    return (np.array(h_array), np.array(s_array))

h_array, s_array = partion_func_results(v_array, T_array, p_array, species)

Results for N2:
------------------------------------------------
| State | Dh' (J mol^-1) | Ds' (J K^-1 mol^-1) |
------------------------------------------------
|   1   |    -632.514    |       -1.888        |
|   2   |    -431.213    |       -1.373        |
------------------------------------------------


# Analytical Results (Different Approach for Task 2)

If you were to solve the departure function analytically (via partial fraction decompositions and substitution), you will achieve the following equations for $\Delta h'$ and $\Delta s'$: 

$$ h_{T,P} - h_{T,P}^{ig} = R T_c \left[ T_r (Z - 1) - (1 + \kappa) \beta  \sqrt{\alpha} \ln \left(\frac{Z + \left( \sqrt{2} + 1 \right) B}{Z - \left(\sqrt{2} - 1 \right) B} \right)\right], $$

$$ s_{T,P} - s_{T,P}^{ig} = R \left[ \ln(Z - B) - \beta \kappa \left( \frac{1 + \kappa}{\sqrt{T_r}} - \kappa \right) \ln \left(\frac{Z + \left( \sqrt{2} + 1 \right) B}{Z - \left(\sqrt{2} - 1 \right) B} \right) \right], $$ 

where 
$$ \beta = \frac{0.45724}{2 (0.07780)\sqrt{2}}, $$
$$ B = 0.07780 \frac{P_r}{T_r},$$
$$ Z = \frac{P v}{R T} $$

Note that this equation is similar to the [wikipedia](https://en.wikipedia.org/wiki/Departure_function) results. The difference here is we used exact values for the departure functions (e.g., Wikipedia page rounds $1 + \sqrt{2} \approx 2.414$ and $\sqrt{2} - 1 \approx 0.414$ rather than computing the exact $1 + \sqrt{2}$ and $\sqrt{2} - 1$ values, respectively).

In [44]:
def h_s_ana(v, T, p, species):
    T_c, P_c, w, MW  = properties(species)
    T_K = T + 273.15
    P_Pa = p * 1e5 # in Pa
    R_gas = 8.314 
    
    # reduce variables
    T_r = T_K / T_c 
    P_r = P_Pa / P_c 
    B = 0.07780 * P_r / T_r
    kappa = 0.37464 + 1.54226 * w - 0.26992 * w**2
    alpha = (1 + kappa * (1 - np.sqrt(T_r)))**2
    constant = (0.45724)/ (2 * np.sqrt(2) * 0.07780) # combine constant of a/(b * 2sqrt(2))
    
    # compressibility
    Z = P_Pa * v / (R_gas * T_K)
    
    # enthalpy 
    log = np.log((Z + (np.sqrt(2) + 1) * B)/(Z - (np.sqrt(2) - 1) * B))
    term1 = constant * (1 + kappa) * np.sqrt(alpha) * log
    term2 = T_r * (Z - 1)
    h = R_gas * T_c * (term2 - term1)
    
    # entropy
    term3 = constant * kappa * ((1 + kappa) / np.sqrt(T_r) - kappa) * log
    term4 = np.log(Z - B)
    s = R_gas * (term4 - term3)
    return (h, s)


def ana_partion_func_results(v, T, p, species):
    """Print results for the entropy and enthalpy departure functions (using analytical method)"""
    h_array = []
    s_array = []
    print(f'Results for {species}:')
    num_lin = 48 * '-'
    print(num_lin)
    print("| State | Dh' (J mol^-1) | Ds' (J K^-1 mol^-1) |")
    print(num_lin)
    for l in range(T.shape[0]):
        h, s = h_s_ana(v[l], T[l], p[l], species)
        h_array += h,
        s_array += s,
        print(f'|{l + 1:^7.0f}|{h:^16.3f}|{s:^21.3f}|')
    print(num_lin)
    return None

ana_partion_func_results(v_array, T_array, p_array, species)

Results for N2:
------------------------------------------------
| State | Dh' (J mol^-1) | Ds' (J K^-1 mol^-1) |
------------------------------------------------
|   1   |    -632.514    |       -1.888        |
|   2   |    -431.213    |       -1.373        |
------------------------------------------------


Numerical and analytical results are agreeing!

# Task 3: Computing Turbine Work and Integrated Heat Capacity

Compute $\displaystyle \int c_p dT$ using the $c_p$ temperature dependent term:

$$ c_p (T) = R\left[ a + bT + \frac{c}{T^2}  \right]$$

Now, we will integrate with respect to $T$ to get: 

$$ \Delta h_{ig} = \int_{T_1}^{T_2} c_p dT = R \left[ a \left( T_2 - T_1 \right) + \frac{b}{2} (T_2^2 - T_1^2) - 
c \left( \frac{1}{T_2} - \frac{1}{T_1} \right) \right]$$

In [45]:
def cpdT(T_1,T_2, species):
    """Integrated ideal heat capacity"""
    R_gas = 8.314
    if species in np.array(['N2']):
        coeff = np.array([3.280,0.593e-3,0.04e5])
    elif species in np.array(['H2O', 'water']):
        coeff = np.array([3.470, 1.45e-3,0.121e5])
    h_ig = R_gas * (coeff[0] * (T_2 - T_1) + (coeff[1]/ 2) * (T_2**2 - T_1**2) - coeff[2] * (1 / T_2 - 1 / T_1))
    return h_ig

def print_cpdT(T, species):
    """Print the integrated ideal heat capacity"""
    print(f'Results for {species}:')
    num_lin = 58 * '-'
    print(num_lin)
    print("| T_1 (C) | T_2 (C) | Integrated Heat Capacity (J / mol) |")
    print(num_lin)
    T_1 = T[0] + 273.15
    T_2 = T[1] + 273.15 
    h_ig = cpdT(T_1, T_2, species)
    print(f'|{T[0]:^9.2f}|{T[1]:^9.2f}|{h_ig:^36.3f}|')
    print(num_lin)
    return h_ig

h_ig = print_cpdT(T_array, species)

Results for N2:
----------------------------------------------------------
| T_1 (C) | T_2 (C) | Integrated Heat Capacity (J / mol) |
----------------------------------------------------------
|  26.85  | -28.15  |             -1598.622              |
----------------------------------------------------------


Now we calculate the work. The work of the turbine is calculated as: 
$$ w_{\text{turbine}} =  - \Delta h_{rg} = -\left( \Delta h_2^{\text{'}} - \Delta h_1^{\text{'}} + \Delta h_{ig} \right)$$

In [46]:
def w_turbine(diff_h1_prim, diff_h2_prim, diff_ig):
    """Calculate the work of a turbine"""
    w_t = -( diff_h2_prim - diff_h1_prim + diff_ig)
    return w_t

def print_w_turbine(T, h1_prim, h2_prim, diff_ig, species):
    """Print the integrated ideal heat capacity"""
    print(f'Results for {species}:')
    num_lin = 47 * '-'
    print(num_lin)
    print("| T_1 (C) | T_2 (C) | Turbine Work (kJ / mol) |")
    print(num_lin)
    T_1 = T[0] + 273.15
    T_2 = T[1] + 273.15 
    w_t = w_turbine(h1_prim, h2_prim, diff_ig)
    print(f'|{T[0]:^9.2f}|{T[1]:^9.2f}|{w_t/1000:^25.3f}|')
    print(num_lin)
    return w_t

w_t = print_w_turbine(T_array, *h_array, h_ig, species)

Results for N2:
-----------------------------------------------
| T_1 (C) | T_2 (C) | Turbine Work (kJ / mol) |
-----------------------------------------------
|  26.85  | -28.15  |          1.397          |
-----------------------------------------------


# Task 4: Isoentropic Temperature and Work

To find the isoentropic temperature, we set the following two equations to equal: 
$$ \int_{T_1}^{T_2} \frac{c_p}{T} dT = R \ln \left( \frac{P_2}{P_1} \right)$$
The integral on the left hand side is:
$$ \int_{T_1}^{T_2} \frac{c_p}{T} dT =  R \left[ a \ln \left( \frac{T_2}{T_1} \right)  + b \left( T_2 - T_1 \right) - \frac{c}{2} \left( \frac{1}{T_2^2} - \frac{1}{T_1^2} \right)\right]$$

We created two functions (one for the left-hand side and the other for the right-hand side). We use fsolve to find $T_2$.

In [47]:
def cp_TdT(T_1, T_2, species): 
    """Computing entropy from Integrated Heat Capacities"""
    R_gas = 8.314
    if species in np.array(['N2']):
        coeff = np.array([3.280, 0.593e-3, 0.04e5])
    elif species in np.array(['H2O','water']):
        coeff = np.array([3.470, 1.45e-3, 0.121e5])
    integration = R_gas * (coeff[0] * np.log(T_2 / T_1) + coeff[1] * (T_2 - T_1) - (coeff[2] / 2) * (1 / T_2**2 - 1 / T_1**2))
    return integration

def s_p(p_1, p_2):
    """Computing the entropy term on the right"""
    R_gas = 8.314
    s = R_gas * np.log(p_2 / p_1)
    return s

def solve_isoentropic(T_2, T_1, p_1, p_2, species):
    """Solve for T for isoentropic"""
    T_1K = T_1 + 273.15
    T_2K = T_2 + 273.15
    lhs = cp_TdT(T_1K, T_2K, species)
    rhs = s_p(p_1, p_2)
    return lhs - rhs

T_trial = T_array[1] 
arguments = (T_array[0], *p_array, species)
T_isoentropic = fsolve(solve_isoentropic,T_trial,arguments)[0]

print(f'Isoentropic exit temperature for {species}: {T_isoentropic:0.2f} C')

Isoentropic exit temperature for N2: -42.34 C


Now we will compute departure functions for enthalpy and entropy (see State 2):

In [48]:
T_iso_array = np.array([T_array[0], T_isoentropic])
h_iso_array, s_iso_array = partion_func_results(v_array, T_iso_array, p_array, species)

Results for N2:
------------------------------------------------
| State | Dh' (J mol^-1) | Ds' (J K^-1 mol^-1) |
------------------------------------------------
|   1   |    -632.514    |       -1.888        |
|   2   |    -453.058    |       -0.908        |
------------------------------------------------


For the ideal gas enthalpy, we get:

In [49]:
h_iso_ig = print_cpdT(T_iso_array, species)

Results for N2:
----------------------------------------------------------
| T_1 (C) | T_2 (C) | Integrated Heat Capacity (J / mol) |
----------------------------------------------------------
|  26.85  | -42.34  |             -2010.505              |
----------------------------------------------------------


Finally, the isoentropic work for the turbine:

In [32]:
w_iso_t = print_w_turbine(T_iso_array, *h_iso_array, h_iso_ig, species)

Results for N2:
-----------------------------------------------
| T_1 (C) | T_2 (C) | Turbine Work (kJ / mol) |
-----------------------------------------------
|  26.85  | -42.34  |          1.831          |
-----------------------------------------------


# Task 5: Efficiency of the Turbine

The efficiency of the turbine is computed as: 
$$ \eta_{\text{turbine}} = \frac{W_{\text{real gas}}}{W_{\text{isoentropic}}} \times 100$$

In [33]:
def eta(W_rg, W_iso):
    """Efficiency of the turbine"""
    efficiency = W_rg / W_iso * 100
    return efficiency

efficiency = eta(w_t, w_iso_t)
print(f'Efficiency of the turbine used: {efficiency:0.2f}%')

Efficiency of the turbine used: 76.31%
